# AM 205 - Midterm

## By Jonathan Guillotte-Blouin

In [28]:
import numpy as np

### Question 1: A stencil based on least squares

#### a) Calculate the coefficients $a_{-1}$, $a_0$, $a_1$, $a_2$, and $a_3$ to make a fourth-order accurate finite-difference formula.

To do this, we will use the Lagrange interpolant approach.

$$ \mathcal{L_{-1}}(z) = \frac{z(z-h)(z-2h)(z-3h)}{(-h)(-2h)(-3h)(-4h)} = \frac{z^4 - 6hz^3 + 11h^2z^2 - 6h^3z}{24h^4}$$

$$ \mathcal{L_{0}}(z) = \frac{(z+h)(z-h)(z-2h)(z-3h)}{h(-h)(-2h)(-3h)} = -\frac{z^4 - 5hz^3 + 5h^2z^2 + 5h^3z - 6h^4}{6h^4}$$

$$ \mathcal{L_{1}}(z) = \frac{(z+h)z(z-2h)(z-3h)}{(2h)h(-h)(-2h)} = \frac{z^4 - 4hz^3 + h^2z^2 + 6h^3z}{4h^4}$$

$$ \mathcal{L_{2}}(z) = \frac{(z+h)z(z-h)(z-3h)}{(3h)(2h)h(-h)} = -\frac{z^4 - 3hz^3 - h^2z^2 + 3h^3z}{6h^4}$$

$$ \mathcal{L_{3}}(z) = \frac{(z+h)z(z-h)(z-2h)}{(4h)(3h)(2h)h} = \frac{z^4 - 2hz^3 - h^2z^2 + 2h^3z}{24h^4}$$

$$ l'(z) = \mathcal{L'_{-1}}(z) \, f_{k-1} + \mathcal{L'_{0}}(z) \, f_k + \mathcal{L'_{1}}(z) \, f_{k+1} + \mathcal{L'_{2}}(z) \, f_{k+2} + \mathcal{L'_{3}}(z) \, f_{k+3} $$

$$ f'_{diff1}(x_k) = l'(0) = \frac{\frac{-1}{4} \, f_{k-1} + \frac{-5}{6} \, f_k  + \frac{3}{2} \, f_{k+1} + \frac{-1}{2} \, f_{k+2} + \frac{1}{12} \, f_{k+3}}{h} $$

#### b) Find the least squares best-fit quadratic through the same points.

In [62]:
def f_prime_diff2(k, xks, yks, N):
    xs = [xks[k-1], xks[k], xks[(k+1) % N], xks[(k+2) % N], xks[(k+3) % N]]
    b = [yks[k-1], yks[k], yks[(k+1) % N], yks[(k+2) % N], yks[(k+3) % N]]
    A = np.fliplr(np.vander(xs, 3))
    _, beta, gamma = np.linalg.lstsq(A, b, rcond=None)[0]
    
    return beta + 2 * gamma * xks[k]

0.3491110975900282

In [56]:
np.fliplr(np.vander([1,2,3,4,5], 3))

array([[ 1,  1,  1],
       [ 1,  2,  4],
       [ 1,  3,  9],
       [ 1,  4, 16],
       [ 1,  5, 25]])

#### c) Which method gives a more accurate result?

$$f'(x_k) = 3\sin(x_k)e^{3\cos(x_k)} \sin\left(e^{3\cos(x_k)}\right)$$

In [66]:
N = 1024
h = 2*np.pi/N
xks = [k*h for k in range(N)]
f_xk = lambda xk: np.cos(np.e**(3*np.cos(xk)))
f_prime_xk = lambda xk: 3 * np.sin(xk) * (np.e**(3*np.cos(xk))) * np.sin(np.e**(3*np.cos(xk)))
fk = [f_xk(xk) for xk in xks]
gk = [fk[i] + 1e-4 * np.random.normal(0, 1) for i in range(N)]

def f_prime_diff1(k, h, fk):
    numerator = (-1/4 * fk[k-1]) + (-5/6 * fk[k]) + (3/2 * fk[(k+1) % N]) + (-1/2 * fk[(k+2) % N]) + (1/12 * fk[(k+3) %N])
    return numerator / h

    

E1, E12, E2, E22 = 0, 0, 0, 0
for k in range(N):
    xk = k*h
    f_prime_result = f_prime_xk(xk)
    
    E1 += (f_prime_result - f_prime_diff1(k, h, fk))**2
    E12 += (f_prime_result - f_prime_diff1(k, h, gk))**2
    
    E2 += (f_prime_result - f_prime_diff2(k, xks, fk, N))**2
    E22 += (f_prime_result - f_prime_diff2(k, xks, gk, N))**2
E1 = np.sqrt(E1/N)
E12 = np.sqrt(E12/N)
E2 = np.sqrt(E2/N)
E22 = np.sqrt(E22/N)

print("E1:", E1)
print("E2:", E2)

E1: 6.646095327302529e-05
E2: 0.018548785431120516


From these results, it looks like the first method (quartic) is more accurate.

#### d) Calculate the new accuracies $E_1$ and $E_2$ using noisy value points $g_k = f_k + 10^{-4}z_k$.

In [67]:
print("E1 (with gk rather than fk):", E12)
print("E2 (with gk rather than fk):", E22)

E1 (with gk rather than fk): 0.03186233642293499
E2 (with gk rather than fk): 0.021227370696174367


From these results using noisy data, it looks like the second method (least squares quadratic) is more accurate.

I think the results from *c)* and *d)* make sense: least squares is an approximation, and so it is understandable that it is less precise than the exact quartic method. However, the exact quartic method does not handle well noisy data (it expects the real data to compute), while least squares is itself an approximation, and so it should be less affected by erronoeus data points.